In [2]:
import gymnasium as gym
import numpy as np
import tensorflow as tf

In [3]:
env = gym.make('ALE/Boxing-v5', render_mode = 'rgb_array', obs_type = 'grayscale')

In [4]:
def crop_image(image):
    return image[30:180, 30:130]

In [5]:
def gen_labels(env):
    labels = np.zeros(18)
    for k in range(18):
        old_state = env.unwrapped.clone_state(include_rng=False)
        labels[k] = env.step(k)[1]
        env.unwrapped.restore_state(old_state)
    return tf.stack(labels.reshape((1, 18)))

In [6]:
def gen_set(num_iter, skip_stride = 5):
    for i in range(num_iter):
        env.reset()
        for j in range(1000):
            step = env.step(env.action_space.sample())
            done = step[2]
            if done:
                break
            image = step[0]
            image = crop_image(image)
            label = gen_labels(env)
            with open('../data/train_data.txt', 'ab') as f:
                np.savetxt(f, image, fmt='%d', delimiter=',')
                np.savetxt(f, label, fmt='%d', delimiter=',')
        with open('../data/train_data.txt', 'ab') as f:
            f.write(b'END_BATCH')


In [7]:
gen_set(1000)